In [1]:
import pandas as pd
from pathlib import Path
import re
import warnings
import csv
import sys
import os
sys.path.append(os.path.join(sys.path[0], '../src'))

from data_preprocessor import preprocess_data

warnings.filterwarnings("ignore")

In [3]:
def print_sample(sample):
    """Print one sample from the entire dataset."""

    print(f"Context:{sample['context']}")
    try:
        print(f"Question:{sample['Question']}")
    except KeyError:
        print(f"Question:{sample['question']}")
    try:
        print(f"Answer:{sample['Answer']}")
    except KeyError:
        print(f"Answer:{sample['answer']}")

In [4]:
BASE_DIR = Path('EDA_MedQuAD.ipynb').resolve().parent.parent
DATA_DIR = BASE_DIR.joinpath('data')
FILE_NAME = "MedQuAD.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [5]:
data = pd.read_csv(FILE, sep=",", index_col=0)

print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285 entries, 0 to 1452
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   context             2285 non-null   object
 1   Question            2285 non-null   object
 2   Answer              2285 non-null   object
 3   Answer_Start_Index  2285 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 89.3+ KB
None


,context,Question,Answer,Answer_Start_Index
0,Описание Кератодермия с пушистыми волосами пре...,Что такое кератодермия с пушистыми волосами?,Кератодермия с пушистыми волосами представляет...,9
1,Описание Кератодермия с пушистыми волосами пре...,Сколько людей с шерстяными волосами страдают к...,Кератодермия с пушистыми волосами встречается ...,9
2,Описание Кератодермия с пушистыми волосами пре...,"Каковы генетические изменения, связанные с кер...","Мутации в генах JUP, DSP, DSC2 и KANK2 вызываю...",2551
3,Описание Кератодермия с пушистыми волосами пре...,Передается ли кератодермия с пушистыми волосами?,Большинство случаев кератодермии с курчавыми в...,4573
4,Описание Кератодермия с пушистыми волосами пре...,Каковы методы лечения кератодермии с пушистыми...,Эти ресурсы посвящены диагностике или лечению ...,-1
5,Описание Синдром Кноблоха — редкое заболевание...,Что такое синдром Кноблоха?,"Синдром Кноблоха — редкое заболевание, характе...",9
6,Описание Синдром Кноблоха — редкое заболевание...,Сколько людей страдает синдромом Кноблоха?,Синдром Кноблоха — редкое заболевание. Однако ...,9
7,Описание Синдром Кноблоха — редкое заболевание...,"Каковы генетические изменения, связанные с син...",Мутации в гене COL18A1 могут вызвать синдром К...,1794
8,Описание Синдром Кноблоха — редкое заболевание...,Передается ли синдром Кноблоха по наследству?,Это состояние наследуется по аутосомно-рецесси...,3141
9,Описание Синдром Кноблоха — редкое заболевание...,Каковы методы лечения синдрома Кноблоха?,Эти ресурсы посвящены диагностике или лечению ...,-1


In [6]:
print_sample(data.iloc[0])

Context:Описание Кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. У людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. В некоторых случаях волосы также редкие. Шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. Начиная с раннего возраста, у пораженных людей также развивается ладонно-подошвенная кератодермия — состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой ​​и мозолистой. Кардиомиопатия, заболевание сердечной мышцы, является опасная для жизни проблема со здоровьем, которая может развиться у людей с кератодермией и пушистыми волосами. В отличие от других особенностей этого состояния, признаки и симптомы кардиомиопатии могут не проявляться до подросткового возраста или позже. Осложнения кардиомиопатии могут включать нарушение

In [7]:
print_sample(data.iloc[10])

Context:Описание Колобома — это глазная аномалия, возникающая еще до рождения. В колобомах отсутствуют кусочки ткани в структурах, формирующих глаз. Они могут выглядеть как выемки или пробелы в одной из нескольких частей глаза, включая цветную часть глаза, называемую радужкой; сетчатка, специализированная светочувствительная ткань, выстилающая заднюю часть глаза; слой кровеносных сосудов под сетчаткой, называемый хориоидеей; или зрительные нервы, которые передают информацию от глаз к мозгу. Колобомы могут присутствовать в одном или обоих глазах и, в зависимости от их размера и местоположения, могут влиять на зрение человека. Колобомы, поражающие радужную оболочку и приводящие к появлению зрачка в виде «замочной скважины», обычно не приводят к потере зрения. Колобомы, поражающие сетчатку, приводят к потере зрения в определенных частях поля зрения. Большие колобомы сетчатки или колобомы, поражающие зрительный нерв, могут вызвать ухудшение зрения, что означает потерю зрения, которую нельз

In [8]:
print_sample(data.iloc[15])

Context:Описание Синдром Lacrimo-auriculo-dento-digital (LADD) — генетическое заболевание, которое в основном поражает глаза, уши, рот и руки. Синдром LADD характеризуется дефектами слезоотделительной системы (лакримо-), проблемами уха (аурикуло-), зубными аномалиями (денто-) и деформациями пальцев (пальцевые). Слезная система состоит из структур глаза. которые производят и выделяют слезы. Пороки развития слезной системы, которые могут возникнуть при синдроме LADD, включают недоразвитие или отсутствие отверстия слезного протока на краю века (слезные точки) и закупорку канала (носослезного канала), соединяющего внутренний угол глаза, где собираются слезы ( слезный мешок) в полость носа. Эти пороки развития слезной системы могут привести к хроническому слезоотделению (эпифоре), воспалению слезного мешка (дакриоциститу), воспалению передней поверхности глаза (кератоконъюнктивиту) или неспособности производить слезы. Низко посаженные и описываемые как чашеобразные, часто сопровождающиеся п

In [9]:
print_sample(data.iloc[100])

Context:Описание Синдром 47,XYY характеризуется наличием дополнительной копии Y-хромосомы в каждой клетке человека. Хотя многие люди с этим заболеванием имеют рост выше среднего, хромосомные изменения иногда не вызывают каких-либо необычных физических особенностей. Большинство людей с синдромом 47,XYY имеют нормальную выработку мужского полового гормона тестостерона и нормальное мужское половое развитие, и они обычно способны стать отцом. Синдром 47,XYY связан с повышенным риском неспособности к обучению и задержкой развития речи и языковые навыки. У больных детей может наблюдаться задержка развития двигательных навыков (например, сидения и ходьбы) или слабый мышечный тонус (гипотония). Другие признаки и симптомы этого состояния включают тремор рук или другие непроизвольные движения (моторные тики), судороги и астму. Лица с синдромом 47,XYY имеют повышенный риск поведенческих, социальных и эмоциональных проблем по сравнению со своими незатронутыми сверстниками. Эти проблемы включают си

In [10]:
data["Answer_Start_Index"].value_counts()

-1       1113
 0        383
 12        65
 20        55
 9         43
         ... 
 2225       1
 413        1
 423        1
 63         1
 1346       1
Name: Answer_Start_Index, Length: 442, dtype: int64

### Data Preprocessing
1. Construct the dataset with columns: ["question", "context", "answer_text", "answer_start"]
2. Check all samples with -1 position
3. Remove all occurences 'Вопрос:' from the "Question" column
4. Remove all occurences 'Резюме:' and 'Краткое описание:' from the "Answer" column
5. Remove all samples with context "На этой странице Основные сведения Начать здесь Диагностика и тесты Узнайте больше Сопутствующие вопросы Подробности Смотрите, играйте и учитесь Изображения Статистика исследований и исследования Клинические испытания Журнальные статьи"
6. Remove all '\n, \t, -, —' and extra spaces
7. Lowercase
8. Remove all duplicates: ["question", "context"]

In [11]:
data = data[["Question", "context", "Answer", "Answer_Start_Index"]].rename(columns={"Question":"question", "Answer":"answer", "Answer_Start_Index":"answer_start"})
data

,question,context,answer,answer_start
0,Что такое кератодермия с пушистыми волосами?,Описание Кератодермия с пушистыми волосами пре...,Кератодермия с пушистыми волосами представляет...,9
1,Сколько людей с шерстяными волосами страдают к...,Описание Кератодермия с пушистыми волосами пре...,Кератодермия с пушистыми волосами встречается ...,9
2,"Каковы генетические изменения, связанные с кер...",Описание Кератодермия с пушистыми волосами пре...,"Мутации в генах JUP, DSP, DSC2 и KANK2 вызываю...",2551
3,Передается ли кератодермия с пушистыми волосами?,Описание Кератодермия с пушистыми волосами пре...,Большинство случаев кератодермии с курчавыми в...,4573
4,Каковы методы лечения кератодермии с пушистыми...,Описание Кератодермия с пушистыми волосами пре...,Эти ресурсы посвящены диагностике или лечению ...,-1
...,...,...,...,...
1448,Вопрос: Что такое дефицит аденозинмонофосфатде...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Дефицит аденозинмонофосфат (АМФ) дезаминазы — ...,12
1449,Вопрос: Сколько людей страдают от дефицита аде...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Дефицит АМФ-дезаминазы является одним из наибо...,174
1450,"Вопрос: Каковы генетические изменения, связанн...",Описание Дефицит аденозинмонофосфат-дезаминазы...,Мутации в гене AMPD1 вызывают дефицит AMP-деза...,1495
1451,Вопрос: Наследуется ли дефицит аденозинмонофос...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Это состояние наследуется по аутосомно-рецесси...,1346


### Check all samples with -1 position

In [12]:
data_tmp = data[data["answer_start"]==-1]
data_tmp

,question,context,answer,answer_start
4,Каковы методы лечения кератодермии с пушистыми...,Описание Кератодермия с пушистыми волосами пре...,Эти ресурсы посвящены диагностике или лечению ...,-1
9,Каковы методы лечения синдрома Кноблоха?,Описание Синдром Кноблоха — редкое заболевание...,Эти ресурсы посвящены диагностике или лечению ...,-1
13,Передается ли колобома по наследству?,"Описание Колобома — это глазная аномалия, возн...",Чаще всего изолированная колобома не передаетс...,-1
29,Каковы методы лечения Т-клеточного иммунодефиц...,"Описание Т-клеточный иммунодефицит, врожденная...",Эти ресурсы посвящены диагностике или лечению ...,-1
32,"Каковы генетические изменения, связанные с син...","Описание Синдром Тимоти — редкое заболевание, ...",Мутации в гене CACNA1C ответственны за все зар...,-1
...,...,...,...,...
1438,Вопрос: Есть ли у вас информация об опасных ма...,"Опасные материалы – это вещества, которые могу...",Краткое описание: Опасные материалы – это веще...,-1
1442,Вопрос: Каковы симптомы гипотермии? (Также наз...,Гипотермия – это опасно низкая температура тел...,"По мере того, как у человека развивается перео...",-1
1444,Вопрос: Что делать при переохлаждении? (Также ...,Гипотермия – это опасно низкая температура тел...,"- НЕ думайте, что человек, найденный неподвижн...",-1
1447,Вопрос: Есть ли у вас информация об измерении ...,Измерение температуры тела может помочь обнару...,Резюме: Измерение температуры тела может помоч...,-1


In [13]:
print_sample(data_tmp.iloc[0])

Context:Описание Кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. У людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. В некоторых случаях волосы также редкие. Шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. Начиная с раннего возраста, у пораженных людей также развивается ладонно-подошвенная кератодермия — состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой ​​и мозолистой. Кардиомиопатия, заболевание сердечной мышцы, является опасная для жизни проблема со здоровьем, которая может развиться у людей с кератодермией и пушистыми волосами. В отличие от других особенностей этого состояния, признаки и симптомы кардиомиопатии могут не проявляться до подросткового возраста или позже. Осложнения кардиомиопатии могут включать нарушение

Remove "-" and "—"

In [14]:
print_sample(data_tmp.iloc[2])

Context:Описание Колобома — это глазная аномалия, возникающая еще до рождения. В колобомах отсутствуют кусочки ткани в структурах, формирующих глаз. Они могут выглядеть как выемки или пробелы в одной из нескольких частей глаза, включая цветную часть глаза, называемую радужкой; сетчатка, специализированная светочувствительная ткань, выстилающая заднюю часть глаза; слой кровеносных сосудов под сетчаткой, называемый хориоидеей; или зрительные нервы, которые передают информацию от глаз к мозгу. Колобомы могут присутствовать в одном или обоих глазах и, в зависимости от их размера и местоположения, могут влиять на зрение человека. Колобомы, поражающие радужную оболочку и приводящие к появлению зрачка в виде «замочной скважины», обычно не приводят к потере зрения. Колобомы, поражающие сетчатку, приводят к потере зрения в определенных частях поля зрения. Большие колобомы сетчатки или колобомы, поражающие зрительный нерв, могут вызвать ухудшение зрения, что означает потерю зрения, которую нельз

In [15]:
print_sample(data_tmp.iloc[1000])

Context:Ваш лечащий врач может попросить вас следить за своим артериальным давлением дома. Для этого вам понадобится домашний тонометр. Выбранный вами монитор должен быть хорошего качества и хорошо подходить по размеру. РУЧНЫЕ МОНИТОРЫ АРТЕРИАЛЬНОГО ДАВЛЕНИЯ. Ручные устройства включают в себя манжету, которая надевается на руку, резиновую грушу и манометр, измеряющий артериальное давление. Для прослушивания пульсации крови в артерии необходим стетоскоп. Вы можете видеть свое кровяное давление на круглом циферблате манометра по мере того, как игла перемещается, а давление в манжетке повышается или падает. При правильном использовании ручные устройства очень полезны. точный. Однако они не являются рекомендуемым типом тонометров для домашнего использования. ЦИФРОВОЙ МОНИТОР АРТЕРИАЛЬНОГО ДАВЛЕНИЯ Цифровое устройство SA также оснащено манжетой, которая надевается на руку. Чтобы надуть манжету, вам может понадобиться резиновый шарик. Другие виды надуваются автоматически при нажатии кнопки. 

In [16]:
print_sample(data_tmp.iloc[400])

Context:На этой странице Основные сведения Начать здесь Диагностика и тесты Профилактика и факторы риска Лечение и терапия Узнайте больше Жизнь с сопутствующими проблемами Особенности Генетика Смотри, играй и учись Изображения Видео и учебные пособия Проверьте свои знания Исследования Клинические испытания Журнальные статьи
Question:Что такое астма у детей?
Answer:Астма — хроническое заболевание, поражающее дыхательные пути. Дыхательные пути представляют собой трубки, по которым воздух поступает в легкие и выходит из них. Если у вас астма, внутренние стенки дыхательных путей становятся болезненными и опухшими. В Соединенных Штатах астмой страдают около 20 миллионов человек. Почти 9 миллионов из них — дети. У детей дыхательные пути меньше, чем у взрослых, что делает астму для них особенно серьезной. Дети, страдающие астмой, могут испытывать одышку, кашель, стеснение в груди и затрудненное дыхание, особенно рано утром или ночью. Многие вещи могут вызвать астму, в том числе - Аллергены - 

Remove all occurences "Резюме"

In [17]:
print_sample(data_tmp.iloc[1000])

Context:Ваш лечащий врач может попросить вас следить за своим артериальным давлением дома. Для этого вам понадобится домашний тонометр. Выбранный вами монитор должен быть хорошего качества и хорошо подходить по размеру. РУЧНЫЕ МОНИТОРЫ АРТЕРИАЛЬНОГО ДАВЛЕНИЯ. Ручные устройства включают в себя манжету, которая надевается на руку, резиновую грушу и манометр, измеряющий артериальное давление. Для прослушивания пульсации крови в артерии необходим стетоскоп. Вы можете видеть свое кровяное давление на круглом циферблате манометра по мере того, как игла перемещается, а давление в манжетке повышается или падает. При правильном использовании ручные устройства очень полезны. точный. Однако они не являются рекомендуемым типом тонометров для домашнего использования. ЦИФРОВОЙ МОНИТОР АРТЕРИАЛЬНОГО ДАВЛЕНИЯ Цифровое устройство SA также оснащено манжетой, которая надевается на руку. Чтобы надуть манжету, вам может понадобиться резиновый шарик. Другие виды надуваются автоматически при нажатии кнопки. 

The answer is not contained in the context

In [18]:
print_sample(data_tmp.iloc[425])

Context:На этой странице Краткое описание основ Начните здесь Симптомы Диагностика и анализы Лечение и методы лечения Узнайте больше Жизнь с сопутствующими проблемами Особенности Генетика Смотрите, играйте и изучайте видео и учебные пособия Статистика исследований и исследования Клинические испытания Журнальные статьи
Question:Что такое болезнь Паркинсона?
Answer:Болезнь Паркинсона (БП) — это тип двигательного расстройства. Это происходит, когда нервные клетки головного мозга не производят достаточного количества химического вещества мозга, называемого дофамином. Иногда это генетическое заболевание, но в большинстве случаев, похоже, оно не передается по наследству. Воздействие химических веществ в окружающей среде может сыграть свою роль. Симптомы начинаются постепенно, часто на одной стороне тела. Позже они затрагивают обе стороны. К ним относятся: - Дрожание кистей, рук, ног, челюсти и лица - Скованность рук, ног и туловища - Медлительность движений - Плохой баланс и координация. По 

There is no meaningful context

In [19]:
# Remove all samples with insignificant context
ids = []
substr = "На этой странице"
for i in range(data.shape[0]):
    if substr in data.iloc[i]["context"]:
        ids.append(i)
data_tmp2 = data.iloc[ids,:]

data_tmp2

,question,context,answer,answer_start
105,Что такое листериозная инфекция?,На этой странице Основные сведения Начать здес...,Листериоз — это заболевание пищевого происхожд...,-1
106,Что такое (являются) низкое кровяное давление?,На этой странице Основные сведения Начни здесь...,"Вы, наверное, слышали, что высокое кровяное да...",-1
107,Есть ли у вас информация об обрезании?,На этой странице Краткое изложение основ Начни...,Краткое описание: Обрезание — это удаление кра...,17
108,Что такое (являются) заболевания толстой кишки?,На этой странице Основные сведения Начать здес...,"Толстая кишка, также известная как толстая киш...",-1
109,Что такое лимфатические заболевания?,На этой странице Основные сведения Начать здес...,Лимфатическая система представляет собой сеть ...,-1
...,...,...,...,...
1397,"Вопрос: Что такое злоупотребление лекарствами,...",На этой странице Основные сведения Начать здес...,"Если вы принимаете лекарство способом, отличны...",-1
1399,Вопрос: Что такое (есть) эректильная дисфункци...,На этой странице Основные сведения Начать здес...,Эректильная дисфункция (ЭД) – распространенный...,-1
1400,Вопрос: Что такое (есть) эректильная дисфункци...,На этой странице Основные сведения Начать здес...,Эректильная дисфункция (ЭД) – распространенный...,-1
1430,Вопрос: Есть ли у вас информация об анализах к...,На этой странице Краткое описание Основы Начни...,"Резюме: Ваша кровь содержит эритроциты (RBC), ...",20


In [20]:
data_tmp2.loc[2228]["context"]

KeyError: 2228

In [ ]:
data_tmp2.loc[2262]["context"]

'На этой странице Краткое описание Основы Начните здесь Узнайте больше Вопросы по теме Особенности Смотрите, играйте и учитесь Изображения Исследования Клинические исследования Журнальные статьи'

In [ ]:
# Remove all samples with insignificant context
ids = []
substr = "Эти ресурсы"
for i in range(data.shape[0]):
    if substr in data.iloc[i]["answer"]:
        ids.append(i)
data_tmp2 = data.iloc[ids,:]

data_tmp2

,question,context,answer,answer_start
4,Каковы методы лечения кератодермии с пушистыми...,Описание Кератодермия с пушистыми волосами пре...,Эти ресурсы посвящены диагностике или лечению ...,-1
9,Каковы методы лечения синдрома Кноблоха?,Описание Синдром Кноблоха — редкое заболевание...,Эти ресурсы посвящены диагностике или лечению ...,-1
14,Каковы методы лечения колобомы?,"Описание Колобома — это глазная аномалия, возн...",Эти ресурсы посвящены диагностике или лечению ...,2156
19,Каковы методы лечения лакримо-аурикуло-денто-п...,Описание Синдром Lacrimo-auriculo-dento-digita...,Эти ресурсы посвящены диагностике или лечению ...,687
24,Каковы методы лечения пиломатрикомы?,"Описание Пиломатриксома, также известная как п...",Эти ресурсы посвящены диагностике и лечению пи...,2030
29,Каковы методы лечения Т-клеточного иммунодефиц...,"Описание Т-клеточный иммунодефицит, врожденная...",Эти ресурсы посвящены диагностике или лечению ...,-1
34,Каковы методы лечения синдрома Тимоти?,"Описание Синдром Тимоти — редкое заболевание, ...",Эти ресурсы посвящены диагностике или лечению ...,3421
39,Каковы методы лечения трисомии 18?,"Описание Трисомия 18, также называемая синдром...",Эти ресурсы посвящены диагностике или лечению ...,-1
44,Каковы методы лечения синдрома Аарскога-Скотта?,Описание Синдром Аарскога-Скотта — это генетич...,Эти ресурсы посвящены диагностике или лечению ...,2337
49,Каковы методы лечения синдрома Паллистера-Холла?,Описание Синдром Паллистера-Холла — заболевани...,Эти ресурсы посвящены диагностике или лечению ...,-1


In [ ]:
data_tmp2.loc[2284]["answer"]

'Эти ресурсы посвящены диагностике или лечению дефицита аденозинмонофосфатдезаминазы: - Энциклопедия MedlinePlus: мышечные боли - Энциклопедия MedlinePlus: слабые стороны Эти ресурсы MedlinePlus предлагают информацию о диагностике и лечении различных состояний здоровья: - Диагностические тесты - Лекарственная терапия - Хирургия и реабилитация - Генетическое консультирование - Паллиативная помощь)'

In [ ]:
data_tmp2.loc[1433]["context"]

'Описание Лейкоэнцефалопатия с исчезновением белого вещества — прогрессирующее заболевание, поражающее преимущественно головной и спинной мозг (центральную нервную систему). Это заболевание вызывает ухудшение состояния белого вещества центральной нервной системы, которое состоит из нервных волокон, покрытых миелином. Миелин — это жировое вещество, которое изолирует и защищает нервы. В большинстве случаев у людей с лейкоэнцефалопатией с исчезающим белым веществом при рождении не наблюдается никаких признаков или симптомов заболевания. У больных детей может наблюдаться небольшая задержка в развитии двигательных навыков, таких как ползание или ходьба. В раннем детстве у большинства больных начинают развиваться двигательные симптомы, включая аномальную ригидность мышц (спастичность) и трудности с координацией движений (атаксия). Также может наблюдаться некоторое ухудшение психического функционирования, но оно обычно не столь выражено, как двигательные симптомы. У некоторых больных женщин м

## Data preprocessing

In [26]:
# Remove extra words
example = data_tmp.iloc[1000]["question"]
print(example.replace('Вопрос:', ''))
example = data_tmp.iloc[1000]["answer"]
print(example.replace('Резюме:', ''))
example = data_tmp.iloc[1100]["answer"]
print(example.replace('Краткое описание:', ''))

# Remove all samples with insignificant context
ids = []
substr = "На этой странице Основные сведения"
for i in range(data_tmp.shape[0]):
    if substr not in data_tmp.iloc[i]["context"]:
        ids.append(i)
data_tmp = data_tmp.iloc[ids,:]


 Есть ли у вас информация о мониторах артериального давления для дома (также называемых: Гипертония - домашний мониторинг)
 Ваш лечащий врач может попросить вас следить за своим кровяным давлением дома. Для этого вам понадобится домашний тонометр. Выбранный вами монитор должен быть хорошего качества и хорошо сидеть. РУЧНЫЕ МОНИТОРЫ АРТЕРИАЛЬНОГО ДАВЛЕНИЯ. Ручные устройства включают в себя манжету, которая надевается на руку, резиновую грушу и манометр, измеряющий артериальное давление. Стетоскоп необходим для прослушивания пульсации крови в артерии. - Вы можете видеть свое артериальное давление на круглом циферблате манометра по мере того, как игла перемещается, а давление в манжете повышается или падает. - При правильном использовании ручные устройства очень точны. Однако они не являются рекомендуемым типом тонометров для домашнего использования. ЦИФРОВЫЕ МОНИТОРЫ АРТЕРИАЛЬНОГО ДАВЛЕНИЯ. Цифровое устройство также будет иметь манжету, которая надевается на руку. Чтобы надуть манжету, в

In [27]:
data_example = " что такое диарея – обзор? также называется: стул водянистый"
data_example = re.sub('[-–—\n\t\(\)\[\]]', '', data_example)
data_example = re.sub(' +', ' ', data_example)
data_example

' что такое диарея обзор? также называется: стул водянистый'

In [28]:
data_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 4 to 1452
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      588 non-null    object
 1   context       588 non-null    object
 2   answer        588 non-null    object
 3   answer_start  588 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 23.0+ KB


In [29]:
# Remove all '\n, \t, -, —' and extra spaces for all columns
example = data_tmp.iloc[20]["question"]
example = re.sub('[-—\n\t\(\)\[\]]', '', example)
example = re.sub(' +', ' ', example)
example

'Что такое являются отравление угарным газом?'

### Data Preprocessing for the entire dataframe

In [30]:
data = preprocess_data(data)
data

,question,context,answer,answer_start
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...,9
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...,9
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю...",2551
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...,4573
5,что такое синдром кноблоха?,"синдром кноблоха — редкое заболевание, характ...","синдром кноблоха — редкое заболевание, характе...",9
...,...,...,...,...
1447,есть ли у вас информация об измерении темпера...,измерение температуры тела может помочь обнару...,измерение температуры тела может помочь обнар...,-1
1448,что такое дефицит аденозинмонофосфатдезаминазы?,дефицит аденозинмонофосфат-дезаминазы (амф) —...,дефицит аденозинмонофосфат (амф) дезаминазы — ...,12
1449,сколько людей страдают от дефицита аденозинмо...,дефицит аденозинмонофосфат-дезаминазы (амф) —...,дефицит амф-дезаминазы является одним из наибо...,174
1450,"каковы генетические изменения, связанные с де...",дефицит аденозинмонофосфат-дезаминазы (амф) —...,мутации в гене ampd1 вызывают дефицит amp-деза...,1495


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1451
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      1460 non-null   object
 1   context       1460 non-null   object
 2   answer        1460 non-null   object
 3   answer_start  1460 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 57.0+ KB


In [32]:
data.groupby(["question", "context"])["answer"].count()

question                                                   context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [33]:
agg = data.groupby(["question", "context"])["answer"].count()

In [34]:
dup_samples = agg[agg>1]
dup_samples

question                                                                                                                   context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [33]:
q = agg.index[1][0]
c = agg.index[1][1]

data[data["question"]==q]

,question,context,answer,answer_start
1060,где найти поддержку людям с витилиго?,"витилиго это состояние кожи, при котором проис...",американский фонд исследования витилиго www.a...,-1
1575,где найти поддержку людям с витилиго?,"витилиго это состояние кожи, при котором проис...",американский фонд исследования витилиго www.a...,-1


In [34]:
q = dup_samples.index[4][0]
c = dup_samples.index[4][1]

data[data["question"]==q].iloc[1]["answer"]

' если у вас рак, вам необходимо хорошее питание, чтобы сохранить ваше тело сильным. для этого вам нужно следить за продуктами, которые вы едите, и за тем, как вы их готовите. некоторые сырые продукты могут содержать микробы, которые могут навредить вам, когда рак или лечение ослабляют вашу иммунную систему. спросите своего врача или медсестру о том, как правильно, но безопасно питаться. когда вы едите яйца: желтки и белки должны быть приготовлены твердыми. не ешьте жидкие яйца. не ешьте продукты, которые могут содержать сырые яйца например, заправку для салата «цезарь», тесто для печенья, тесто для торта и голландский соус. будьте осторожны с молочными продуктами: на упаковке любого молока, йогурта, сыра и других молочных продуктов должно быть написано слово «пастеризовано». не ешьте мягкие сыры или сыры с голубыми прожилками например, бри, камамбер, рокфор, стилтон, горгонзола и блю. не ешьте сыры в мексиканском стиле например, кесо бланко фреска и кохита. фрукты и овощи: мойте все с

In [35]:
q = dup_samples.index[3][0]
c = dup_samples.index[3][1]

data[data["question"]==q]

,question,context,answer,answer_start
1100,есть ли у вас информация о безопасном использ...,"лекарства, отпускаемые без рецепта otc, это ле...","лекарства, отпускаемые без рецепта otc, это л...",-1
1315,есть ли у вас информация о безопасном использ...,"лекарства, отпускаемые без рецепта otc, это ле...","лекарства, отпускаемые без рецепта otc, это л...",-1
1469,есть ли у вас информация о безопасном использ...,"лекарства, отпускаемые без рецепта otc, это ле...","лекарства, отпускаемые без рецепта otc, это л...",-1
1530,есть ли у вас информация о безопасном использ...,"лекарства, отпускаемые без рецепта otc, это ле...","лекарства, отпускаемые без рецепта otc, это л...",-1
1751,есть ли у вас информация о безопасном использ...,"лекарства, отпускаемые без рецепта otc, это ле...","лекарства, отпускаемые без рецепта otc, это л...",-1


In [35]:
# Remove duplicates
data.drop_duplicates(subset=['question', 'context'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1113 entries, 0 to 1451
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      1113 non-null   object
 1   context       1113 non-null   object
 2   answer        1113 non-null   object
 3   answer_start  1113 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 43.5+ KB


### Save Dataset

In [37]:
DATA_NAME = "MedQuAD_clean.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)

In [38]:
data["context"]

0        кератодермия с пушистыми волосами представляе...
1        кератодермия с пушистыми волосами представляе...
2        кератодермия с пушистыми волосами представляе...
3        кератодермия с пушистыми волосами представляе...
5        синдром кноблоха редкое заболевание, характер...
                              ...                        
2279    измерение температуры тела может помочь обнару...
2280     дефицит аденозинмонофосфатдезаминазы амф это ...
2281     дефицит аденозинмонофосфатдезаминазы амф это ...
2282     дефицит аденозинмонофосфатдезаминазы амф это ...
2283     дефицит аденозинмонофосфатдезаминазы амф это ...
Name: context, Length: 1113, dtype: object

In [40]:
len(set(data["context"]))

593

In [56]:
data_unique_context = pd.DataFrame(set(data["context"]))

data_unique_context.to_csv(DATA_DIR.joinpath("unique_context.csv"))

# with open(DATA_DIR.joinpath("unique_context.csv"), 'w', newline='') as file:
#     writer = csv.writer(file)
#     for row in data_unique_context[0]:
#         writer.writerow(row)